In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("combined.csv")
df.rename(columns={"language": "Language", "runtime": "Runtime", "energy_usage (sum)": "Energy Usage", "execution_time (sum)": "Execution Time", "memory_usage (sum)": "Memory Usage", "cpu_usage (avg)": "CPU Usage"}, inplace=True)
df_no_id = df.drop(labels="id", axis=1)
df_no_id

,Language,Runtime,Energy Usage,Execution Time,Memory Usage,CPU Usage
0,c,wasmer,1779.577,929650,1.900,99.279667
1,c,wasmer,1779.990,929640,2.000,99.589667
2,c,wasmer,1778.562,929620,1.900,99.200667
3,c,wasmer,1780.095,929850,2.399,99.364667
4,c,wasmer,1780.253,929460,1.898,99.151333
5,c,wasmer,1779.858,929920,2.499,99.431000
6,c,wasmer,1779.961,929820,1.900,99.752667
7,c,wasmer,1780.518,929800,1.998,99.450333
8,c,wasmer,1780.101,930030,2.498,99.819000
9,c,wasmer,1778.983,929670,2.000,99.880333


In [3]:
# RQ 1: only runtime wasmer and then 3 languages
def make_table(df : pd.DataFrame, runtime : str) -> pd.DataFrame:
    df_c = df[(df["Runtime"] == runtime) & (df_no_id["Language"] == "c")].describe()
    df_go = df[(df["Runtime"] == runtime) & (df_no_id["Language"] == "go")].describe()
    df_rust = df[(df["Runtime"] == runtime) & (df_no_id["Language"] == "rust")].describe()

    a = df_c.add_prefix("C_")
    b = df_rust.add_prefix("Rust_")
    c = df_go.add_prefix("Go_")
    d = pd.concat([a,b,c], axis=1)
    d.columns = pd.MultiIndex.from_tuples([tuple(s.split("_")) for s in d.columns])

    return d

In [4]:
def rq_13(df : pd.DataFrame, runtime : str, column : str) -> pd.DataFrame:
    df_c = df[(df["Runtime"] == runtime) & (df_no_id["Language"] == "c")].rename(columns={column: "C"}).describe()["C"]
    df_go = df[(df["Runtime"] == runtime) & (df_no_id["Language"] == "go")].rename(columns={column: "Go"}).describe()["Go"]
    df_rust = df[(df["Runtime"] == runtime) & (df_no_id["Language"] == "rust")].rename(columns={column: "Rust"}).describe()["Rust"]

    d = pd.concat([df_c,df_go,df_rust], axis=1)

    return d

In [5]:
def make_table_metric(df : pd.DataFrame, metric : str) -> pd.DataFrame:
    df_wasmer = rq_13(df, "wasmer", metric)
    df_wasmtime = rq_13(df, "wasmtime", metric)

    a = df_wasmer.add_prefix("Wasmer_")
    b = df_wasmtime.add_prefix("Wasmtime_")

    d = pd.concat([a,b], axis=1)
    d.columns = pd.MultiIndex.from_tuples([tuple(s.split("_")) for s in d.columns])
    return d

In [6]:
# make table RQ1 + 2
parameter_name = "Energy Usage"
label = f"tab:rq12_energyUsage"
b = rq_13(df_no_id, "wasmer", "Energy Usage")
caption = f"Descriptive statistics for RQ1 and RQ2 - {parameter_name}."
b.to_latex("rq12.tex",column_format=">{\\bfseries}c" + len(b.columns) * "c", multicolumn_format="c", caption=caption, label=label)

/tmp/ipykernel_133134/3038136253.py:6: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  b.to_latex("rq12.tex",column_format=">{\\bfseries}c" + len(b.columns) * "c", multicolumn_format="c", caption=caption, label=label)


In [7]:
keys = ["Energy Usage", "Execution Time", "Memory Usage", "CPU Usage"]
for k in keys:
    parameter_name = k
    label = f"tab:rq_{parameter_name}"
    b = make_table_metric(df_no_id, parameter_name)
    caption = f"Descriptive statistics for RQ3 and RQ4 - {parameter_name}."
    b.to_latex(f"rq_{parameter_name}.tex",column_format=">{\\bfseries}c" + len(b.columns) * "c", multicolumn_format="c", caption=caption, label=label)

/tmp/ipykernel_133134/3618195737.py:7: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  b.to_latex(f"rq_{parameter_name}.tex",column_format=">{\\bfseries}c" + len(b.columns) * "c", multicolumn_format="c", caption=caption, label=label)
/tmp/ipykernel_133134/3618195737.py:7: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  b.to_latex(f"rq_{parameter_name}.tex",column_format=">{\\bfseries}c" + len(b.columns) * "c", multicolumn_format="c", caption=caption, label=label)
/tmp/ipykernel

In [8]:
make_table_metric(df_no_id, "Energy Usage")

Wasmer                               Wasmtime               \
                 C           Go         Rust            C           Go   
count    10.000000    10.000000    10.000000    10.000000    10.000000   
mean   1779.789800  3406.947700  1100.730300  1702.376100  3733.013900   
std       0.597454     4.516328     0.716933     0.903716     6.373089   
min    1778.562000  3402.592000  1100.279000  1701.351000  3727.629000   
25%    1779.647250  3404.481250  1100.426500  1701.609500  3728.852000   
50%    1779.975500  3405.523000  1100.492500  1702.190000  3730.050500   
75%    1780.099500  3406.425500  1100.629500  1703.036250  3736.474250   
max    1780.518000  3415.520000  1102.698000  1703.764000  3744.696000   

                    
              Rust  
count    10.000000  
mean   1116.511900  
std       1.265197  
min    1115.238000  
25%    1115.742750  
50%    1115.811000  
75%    1117.428250  
max    1119.014000